In [1]:
import pandas
from nltk import ngrams


def tokenize(text, nGram=3):
    text = text.split(" ")
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(x, nGram):
  overlaps = []
  for h, r in zip([x["texto_x"]], [x["texto_y"]]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, nGram)
    b = tokenize(r, nGram)

    if len(a) >= len(b) and len(a) > 0:
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a) and len(b) > 0:
      overlaps.append(len(set(a) & set(b))/len(b))
    elif len(a) == 0 or len(b) == 0:
      overlaps.append(0)

  return overlaps[0]

def modulus(x):
    if x >= 0:
        return x
    elif x < 0: 
        return -(x)

In [2]:
df = pandas.read_parquet("./data/complete.parquet")

In [3]:
###Formal -> informal


df_formal   = df[df["formal/informal_x"] == "formal"]
df_informal = df[df["formal/informal_x"] == "informal"]

df_formal_informal = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "forma_x_x", "linguagem_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "forma_x_y", "linguagem_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "formal/informal_x_y": "formal/informal_y",
                                                                                                                                                                               "formal/informal_x_x": "formal/informal_x",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

###arcaica -> moderna

df_formal   = df[df["linguagem_x"] == "arcaica"]
df_informal = df[df["linguagem_x"] == "moderna"]

df_arcaica_moderna = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "forma_x_x", "formal/informal_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "forma_x_y", "formal/informal_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "linguagem_x_y": "linguagem_y",
                                                                                                                                                                               "linguagem_x_x": "linguagem_x",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})
###literal -> dinâmica

df_formal   = df[df["forma_x"] == "literal"]
df_informal = df[df["forma_x"] == "dinâmica"]

df_literal_dinamica = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "linguagem_x_x", "formal/informal_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "linguagem_x_y", "formal/informal_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "forma_x_x": "forma_x",
                                                                                                                                                                               "forma_x_y": "forma_y",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

In [4]:
df_literal_dinamica["overlap-3"] = df_literal_dinamica.apply(lambda x: getNgramOverlap(x, 3), axis=1)
df_literal_dinamica["diferenca"] = (df_literal_dinamica["sourceLen"] - df_literal_dinamica["targetLen"]).apply(lambda x: modulus(x))

df_arcaica_moderna["overlap-3"] = df_arcaica_moderna.apply(lambda x: getNgramOverlap(x, 3), axis=1)
df_arcaica_moderna["diferenca"] = (df_arcaica_moderna["sourceLen"] - df_arcaica_moderna["targetLen"]).apply(lambda x: modulus(x))

df_formal_informal["overlap-3"] = df_formal_informal.apply(lambda x: getNgramOverlap(x, 3), axis=1)
df_formal_informal["diferenca"] = (df_formal_informal["sourceLen"] - df_formal_informal["targetLen"]).apply(lambda x: modulus(x))



In [5]:
pandas.set_option('display.max_colwidth', None)

In [61]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] <= 0.01) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])




52137    têm um versículo para cada uma das
Name: texto_x, dtype: object
52137    As estradas que levam a Sião estão tristes , pois não há ninguém que vá por elas para as festas religiosas . As moças que cantavam no Templo estão aflitas , e os sacerdotes vivem gemendo . A cidade sofre amargamente , e não há gente para se reunir nas suas praças .
Name: texto_y, dtype: object
52137    0.009524
Name: overlap-3, dtype: float64


In [63]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.01) & (df_formal_informal["overlap-3"] <= 0.10) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])


67405    a mim que anteriormente fui blasfemo , perseguidor e insolente ; mas alcancei misericórdia , porque o fiz por ignorância e na minha incredulidade ;
Name: texto_x, dtype: object
67405    Ele fez isso apesar de eu ter dito blasfêmias contra ele no passado e de o ter perseguido e insultado . Mas Deus teve misericórdia de mim , pois eu não tinha fé e por isso não sabia o que estava fazendo .
Name: texto_y, dtype: object
67405    0.082353
Name: overlap-3, dtype: float64


In [64]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.10) & (df_formal_informal["overlap-3"] <= 0.20) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

3921    Eis que o Senhor , teu Deus , te deu esta terra diante de ti ; sobe e possui-a como te falou o Senhor , Deus de teus pais ; não temas e não te assustes .
Name: texto_x, dtype: object
3921    Portanto , vão e tomem posse dessa terra que está diante de vocês , como o Senhor , o Deus dos nossos antepassados , mandou . Não tenham medo , nem se assustem. ”
Name: texto_y, dtype: object
3921    0.191781
Name: overlap-3, dtype: float64


In [65]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.20) & (df_formal_informal["overlap-3"] <= 0.30) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

53777    Nomearam reis sem que eu consentisse e escolheram príncipes sem minha aprovação . Fizeram para si ídolos de prata e ouro e assim provocaram a própria destruição .
Name: texto_x, dtype: object
53777    — O meu povo escolheu reis sem me consultar e nomeou governadores sem a minha aprovação . Com a sua prata e com o seu ouro , fizeram ídolos e por isso serão destruídos .
Name: texto_y, dtype: object
53777    0.202899
Name: overlap-3, dtype: float64


In [67]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.30) & (df_formal_informal["overlap-3"] <= 0.40) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

22461    “ Naquela ocasião ordenei aos seus juízes : Atendam as demandas de seus irmãos e julguem com justiça , não só as questões entre os seus compatriotas mas também entre um israelita e um estrangeiro .
Name: texto_x, dtype: object
22461    Naquela mesma ocasião dei a seguinte ordem aos juízes : “ Julguem todas as causas com justiça , seja entre dois israelitas , seja entre um israelita e um estrangeiro que vive no meio do povo .
Name: texto_y, dtype: object
22461    0.328767
Name: overlap-3, dtype: float64


In [68]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.40) & (df_formal_informal["overlap-3"] <= 0.50) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

70166    Fará muitos israelitas voltarem ao Senhor , seu Deus .
Name: texto_x, dtype: object
70166    e levará muitos israelitas ao Senhor , o Deus de Israel .
Name: texto_y, dtype: object
70166    0.434783
Name: overlap-3, dtype: float64


In [69]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.50) & (df_formal_informal["overlap-3"] <= 0.60) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

70296    Vinho novo deve ser guardado em recipientes novos .
Name: texto_x, dtype: object
70296    Não . Vinho novo deve ser posto em odres novos .
Name: texto_y, dtype: object
70296    0.52381
Name: overlap-3, dtype: float64


In [70]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.60) & (df_formal_informal["overlap-3"] <= 0.70) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

64679    Respondeu Simão Pedro : “ Então , Senhor , não apenas os meus pés , mas também as minhas mãos e a minha cabeça ! ”
Name: texto_x, dtype: object
64679    — Então , Senhor , não lave somente os meus pés ; lave também as minhas mãos e a minha cabeça ! — pediu Simão Pedro .
Name: texto_y, dtype: object
64679    0.622642
Name: overlap-3, dtype: float64


In [71]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.70) & (df_formal_informal["overlap-3"] <= 0.80) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

68234    Nós amamos porque ele nos amou primeiro .
Name: texto_x, dtype: object
68234    Nós amamos porque Deus nos amou primeiro .
Name: texto_y, dtype: object
68234    0.8
Name: overlap-3, dtype: float64


In [73]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.80) & (df_formal_informal["overlap-3"] <= 0.90) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

30610    Odeio e detesto a falsidade , mas amo a tua lei .
Name: texto_x, dtype: object
30610    Odeio e detesto a mentira , mas amo a tua lei .
Name: texto_y, dtype: object
30610    0.826087
Name: overlap-3, dtype: float64


In [74]:
sample = df_formal_informal[(df_formal_informal["overlap-3"] > 0.90) & (df_formal_informal["overlap-3"] <= 1.00) & (df_formal_informal["language"] == "PT")].sample(1)

print(sample["texto_x"])
print(sample["texto_y"])
print(sample["overlap-3"])

12265    O meu socorro vem do Senhor , que fez o céu e a terra .
Name: texto_x, dtype: object
12265    O meu socorro vem do Senhor Deus , que fez o céu e a terra .
Name: texto_y, dtype: object
12265    0.903226
Name: overlap-3, dtype: float64


antes:  205067933
depois:  204899044


In [6]:



print("letral - dinamica")

print("antes: ", df_literal_dinamica.shape)
print("depois: ", df_literal_dinamica[(df_literal_dinamica["overlap-3"] > 0.01)].shape)

print("antes: ", df_literal_dinamica["sourceLen"].sum() + df_literal_dinamica["targetLen"].sum())
print("depois: ", df_literal_dinamica[(df_literal_dinamica["overlap-3"] > 0.01)]["sourceLen"].sum() + df_literal_dinamica[(df_literal_dinamica["overlap-3"] > 0.01)]["targetLen"].sum())

print("antes: ", (df_literal_dinamica["sourceLen"].mean() + df_literal_dinamica["targetLen"].mean())/2)
print("depois: ", (df_literal_dinamica[(df_literal_dinamica["overlap-3"] > 0.01)]["sourceLen"].mean() + df_literal_dinamica[(df_literal_dinamica["overlap-3"] > 0.01)]["targetLen"].mean()) / 2)

print("------")

print("formal - informal")
print("antes: ", df_formal_informal.shape)
print("depois: ", df_formal_informal[(df_formal_informal["overlap-3"] > 0.01)].shape)

print("antes: ", df_formal_informal["sourceLen"].sum() + df_formal_informal["targetLen"].sum())
print("depois: ", df_formal_informal[(df_formal_informal["overlap-3"] > 0.01)]["sourceLen"].sum() + df_formal_informal[(df_formal_informal["overlap-3"] > 0.01)]["targetLen"].sum())

print("antes: ", (df_formal_informal["sourceLen"].mean() + df_formal_informal["targetLen"].mean())/2)
print("depois: ", (df_formal_informal[(df_formal_informal["overlap-3"] > 0.01)]["sourceLen"].mean() + df_formal_informal[(df_formal_informal["overlap-3"] > 0.01)]["targetLen"].mean()) / 2)

print("------")

print("arcaica - moderna")
print("antes: ", df_arcaica_moderna.shape)
print("depois: ", df_arcaica_moderna[(df_arcaica_moderna["overlap-3"] > 0.01)].shape)


print("antes: ", df_arcaica_moderna["sourceLen"].sum() + df_arcaica_moderna["targetLen"].sum())
print("depois: ", df_arcaica_moderna[(df_arcaica_moderna["overlap-3"] > 0.01)]["sourceLen"].sum() + df_arcaica_moderna[(df_arcaica_moderna["overlap-3"] > 0.01)]["targetLen"].sum())


print("antes: ", (df_arcaica_moderna["sourceLen"].mean() + df_arcaica_moderna["targetLen"].mean())/2)
print("depois: ", (df_arcaica_moderna[(df_arcaica_moderna["overlap-3"] > 0.01)]["sourceLen"].mean() + df_arcaica_moderna[(df_arcaica_moderna["overlap-3"] > 0.01)]["targetLen"].mean()) / 2)


letral - dinamica
antes:  (3173858, 12)
depois:  (3169293, 12)
antes:  205067933
depois:  204899044
antes:  32.30578258384591
depois:  32.32567074107695
------
formal - informal
antes:  (3218423, 12)
depois:  (3212611, 12)
antes:  200933734
depois:  200732567
antes:  31.216178544585347
depois:  31.24134341194748
------
arcaica - moderna
antes:  (2911032, 12)
depois:  (2906935, 12)
antes:  189930304
depois:  189775582
antes:  32.62250363444991
depois:  32.64186884123656


In [7]:
df_literal_dinamica = df_literal_dinamica[(df_literal_dinamica["overlap-3"] > 0.01)]
df_formal_informal = df_formal_informal[(df_formal_informal["overlap-3"] > 0.01)]
df_arcaica_moderna = df_arcaica_moderna[(df_arcaica_moderna["overlap-3"] > 0.01)]

df_arcaica_moderna.to_parquet("./data/arcaico_moderno.parquet", index=False)
df_formal_informal.to_parquet("./data/formal_informal.parquet", index=False)
df_literal_dinamica.to_parquet("./data/literal_dinamica.parquet", index=False)